In [1]:
import os
import random
import shutil
import time
import warnings
from datetime import datetime
import argparse

import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim

from torch.cuda.amp import autocast, GradScaler

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('logs')

SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

GPU = 'cuda:0'
START_EPOCH = 0
ARCH = 'densenet'
EPOCHS = 10
LR = .0005
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
BATCH_SIZE = 100
WORKERS=2
#TRAINDIR="data/training/DEWP_class"
#VALDIR="data/test/DEWP_class"
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [2]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def train(train_loader, model, criterion, optimizer, scaler, epoch):

    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top2 = AverageMeter('Acc@2', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top2],
        prefix="Epoch: [{}]".format(epoch))

    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
                
        images = images.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        optimizer.zero_grad()
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        acc1, acc2 = accuracy(output, target, topk=(1, 2))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top2.update(acc2[0], images.size(0))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % PRINT_FREQ == 0:
            progress.display(i)

            writer.add_scalar('Train Acc@1',
                acc1[0],
                epoch * len(train_loader) + i)

def validate(val_loader, model, criterion, epoch):

    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top2 = AverageMeter('Acc@2', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top2],
        prefix='Test: ')

    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            images = images.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)
            
            output = model(images)
            loss = criterion(output, target)

            acc1, acc2 = accuracy(output, target, topk=(1, 2))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top2.update(acc2[0], images.size(0))
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

                writer.add_scalar('Validation Acc@1',
                    acc1[0],
                    epoch * len(val_loader) + i)

def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)

In [3]:
##From PyTorch docs https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html 

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [4]:
def single_init(train_dir, val_dir, num_classes):
    model_ft, IMG_SIZE = initialize_model(ARCH, num_classes, False, use_pretrained=False)

    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB)
    ])

    transform_val = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB)
    ])

    torch.cuda.set_device(GPU)
    model_ft.cuda(GPU)
    criterion = nn.CrossEntropyLoss().cuda(GPU)
    scaler = GradScaler()

    optimizer = optim.SGD(
        model_ft.parameters(),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
        momentum=MOMENTUM
    )
    
    model = model_ft

    # use CosineAnnealingLR
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=EPOCHS)
    
    train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)

    val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transform_val)
    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True) 

    return (model, criterion, optimizer, scheduler, scaler, train_loader, val_loader)

In [5]:
TEMP_CHECKPOINT_PATH = "./checkpoint_temp.pth.tar"
DEWP_CHECKPOINT_PATH = "./checkpoint_dewp.pth.tar"
VISIB_CHECKPOINT_PATH = "./checkpoint_visib.pth.tar"
WDSP_CHECKPOINT_PATH = "./checkpoint_wdsp.pth.tar"

TEMP_training_dir = "../../../data/training/TEMP_class"
TEMP_test_dir = "../../../data/test/TEMP_class"
DEWP_training_dir = "../../../data/training/DEWP_class"
DEWP_test_dir = "../../../data/test/DEWP_class"
VISIB_training_dir = "../../../data/training/VISIB_class"
VISIB_test_dir = "../../../data/test/VISIB_class"
WDSP_training_dir = "../../../data/training/WDSP_class"
WDSP_test_dir = "../../../data/test/WDSP_class"

In [6]:
model, criterion, optimizer, scheduler, scaler, train_loader, val_loader = single_init(DEWP_training_dir, DEWP_test_dir, num_classes=3)
    
start = datetime.now()

for epoch in range(START_EPOCH, EPOCHS):

    train(train_loader, model, criterion, optimizer, scaler, epoch)
    validate(val_loader, model, criterion, epoch)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, filename=DEWP_CHECKPOINT_PATH)

    scheduler.step()
    print('Learning rate: ' + str(scheduler.get_last_lr()))
    print("Time elapsed: " + str(datetime.now() - start))

/home/ec2-user/data/venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/data/venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch: [0][  0/980]	Time  5.561 ( 5.561)	Data  0.000 ( 0.000)	Loss 1.1840e+00 (1.1840e+00)	Acc@1  31.00 ( 31.00)	Acc@2  55.00 ( 55.00)
Epoch: [0][ 50/980]	Time  0.857 ( 0.962)	Data  0.000 ( 0.000)	Loss 1.0413e+00 (1.0772e+00)	Acc@1  50.00 ( 43.78)	Acc@2  77.00 ( 75.39)
Epoch: [0][100/980]	Time  0.893 ( 0.919)	Data  0.000 ( 0.000)	Loss 1.0331e+00 (1.0704e+00)	Acc@1  44.00 ( 43.82)	Acc@2  83.00 ( 76.68)
Epoch: [0][150/980]	Time  0.875 ( 0.904)	Data  0.000 ( 0.000)	Loss 1.0572e+00 (1.0661e+00)	Acc@1  45.00 ( 44.07)	Acc@2  80.00 ( 77.44)
Epoch: [0][200/980]	Time  0.877 ( 0.897)	Data  0.000 ( 0.000)	Loss 1.0855e+00 (1.0650e+00)	Acc@1  37.00 ( 43.69)	Acc@2  76.00 ( 77.67)
Epoch: [0][250/980]	Time  0.879 ( 0.893)	Data  0.000 ( 0.000)	Loss 1.0677e+00 (1.0641e+00)	Acc@1  46.00 ( 43.59)	Acc@2  75.00 ( 77.78)
Epoch: [0][300/980]	Time  0.864 ( 0.890)	Data  0.000 ( 0.000)	Loss 1.0398e+00 (1.0639e+00)	Acc@1  50.00 ( 43.47)	Acc@2  77.00 ( 77.78)
Epoch: [0][350/980]	Time  0.876 ( 0.889)	Data  0.000 ( 

In [7]:
#Temp model

model, criterion, optimizer, scheduler, scaler, train_loader, val_loader = single_init(TEMP_training_dir, TEMP_test_dir, num_classes=4)
    
start = datetime.now()

for epoch in range(START_EPOCH, EPOCHS):

    train(train_loader, model, criterion, optimizer, scaler, epoch)
    validate(val_loader, model, criterion, epoch)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, filename=TEMP_CHECKPOINT_PATH)

    scheduler.step()
    print('Learning rate: ' + str(scheduler.get_last_lr()))
    print("Time elapsed: " + str(datetime.now() - start))

Epoch: [0][  0/991]	Time  0.878 ( 0.878)	Data  0.000 ( 0.000)	Loss 1.4367e+00 (1.4367e+00)	Acc@1  17.00 ( 17.00)	Acc@2  44.00 ( 44.00)
Epoch: [0][ 50/991]	Time  0.883 ( 0.875)	Data  0.000 ( 0.000)	Loss 1.2323e+00 (1.3346e+00)	Acc@1  52.00 ( 40.04)	Acc@2  73.00 ( 61.82)
Epoch: [0][100/991]	Time  0.896 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.2639e+00 (1.3253e+00)	Acc@1  48.00 ( 41.02)	Acc@2  71.00 ( 63.24)
Epoch: [0][150/991]	Time  0.874 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.2531e+00 (1.3189e+00)	Acc@1  48.00 ( 41.67)	Acc@2  65.00 ( 63.54)
Epoch: [0][200/991]	Time  0.871 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.3600e+00 (1.3178e+00)	Acc@1  37.00 ( 41.75)	Acc@2  63.00 ( 63.64)
Epoch: [0][250/991]	Time  0.873 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.3320e+00 (1.3160e+00)	Acc@1  40.00 ( 41.92)	Acc@2  61.00 ( 63.78)
Epoch: [0][300/991]	Time  0.871 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.3711e+00 (1.3145e+00)	Acc@1  37.00 ( 42.03)	Acc@2  61.00 ( 63.94)
Epoch: [0][350/991]	Time  0.887 ( 0.877)	Data  0.000 ( 

In [8]:
#Visib model

model, criterion, optimizer, scheduler, scaler, train_loader, val_loader = single_init(VISIB_training_dir, VISIB_test_dir, num_classes=3)
    
start = datetime.now()

for epoch in range(START_EPOCH, EPOCHS):

    train(train_loader, model, criterion, optimizer, scaler, epoch)
    validate(val_loader, model, criterion, epoch)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, filename=VISIB_CHECKPOINT_PATH)

    scheduler.step()
    print('Learning rate: ' + str(scheduler.get_last_lr()))
    print("Time elapsed: " + str(datetime.now() - start))

Epoch: [0][  0/879]	Time  0.887 ( 0.887)	Data  0.000 ( 0.000)	Loss 1.1702e+00 (1.1702e+00)	Acc@1  18.00 ( 18.00)	Acc@2  54.00 ( 54.00)
Epoch: [0][ 50/879]	Time  0.878 ( 0.879)	Data  0.000 ( 0.000)	Loss 1.0296e+00 (1.0152e+00)	Acc@1  50.00 ( 49.73)	Acc@2  83.00 ( 82.80)
Epoch: [0][100/879]	Time  0.877 ( 0.877)	Data  0.000 ( 0.000)	Loss 1.0858e+00 (9.9516e-01)	Acc@1  47.00 ( 51.97)	Acc@2  78.00 ( 84.46)
Epoch: [0][150/879]	Time  0.880 ( 0.876)	Data  0.000 ( 0.000)	Loss 9.6599e-01 (9.8970e-01)	Acc@1  52.00 ( 52.29)	Acc@2  88.00 ( 84.99)
Epoch: [0][200/879]	Time  0.876 ( 0.877)	Data  0.000 ( 0.000)	Loss 9.2504e-01 (9.8469e-01)	Acc@1  56.00 ( 52.70)	Acc@2  91.00 ( 85.31)
Epoch: [0][250/879]	Time  0.878 ( 0.877)	Data  0.000 ( 0.000)	Loss 9.7063e-01 (9.7862e-01)	Acc@1  62.00 ( 53.24)	Acc@2  82.00 ( 85.69)
Epoch: [0][300/879]	Time  0.879 ( 0.878)	Data  0.000 ( 0.000)	Loss 9.8341e-01 (9.7685e-01)	Acc@1  53.00 ( 53.23)	Acc@2  87.00 ( 85.86)
Epoch: [0][350/879]	Time  0.871 ( 0.878)	Data  0.000 ( 

In [9]:
#WDSP model

model, criterion, optimizer, scheduler, scaler, train_loader, val_loader = single_init(WDSP_training_dir, WDSP_test_dir, num_classes=3)
    
start = datetime.now()

for epoch in range(START_EPOCH, EPOCHS):

    train(train_loader, model, criterion, optimizer, scaler, epoch)
    validate(val_loader, model, criterion, epoch)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, filename=WDSP_CHECKPOINT_PATH)

    scheduler.step()
    print('Learning rate: ' + str(scheduler.get_last_lr()))
    print("Time elapsed: " + str(datetime.now() - start))

Epoch: [0][  0/975]	Time  0.892 ( 0.892)	Data  0.000 ( 0.000)	Loss 1.1226e+00 (1.1226e+00)	Acc@1  24.00 ( 24.00)	Acc@2  65.00 ( 65.00)
Epoch: [0][ 50/975]	Time  0.878 ( 0.881)	Data  0.000 ( 0.000)	Loss 1.0258e+00 (1.0571e+00)	Acc@1  55.00 ( 46.98)	Acc@2  76.00 ( 77.39)
Epoch: [0][100/975]	Time  0.881 ( 0.879)	Data  0.000 ( 0.000)	Loss 1.1046e+00 (1.0570e+00)	Acc@1  34.00 ( 46.96)	Acc@2  76.00 ( 77.09)
Epoch: [0][150/975]	Time  0.885 ( 0.879)	Data  0.000 ( 0.000)	Loss 1.0900e+00 (1.0553e+00)	Acc@1  43.00 ( 47.36)	Acc@2  77.00 ( 77.13)
Epoch: [0][200/975]	Time  0.874 ( 0.880)	Data  0.000 ( 0.000)	Loss 1.0991e+00 (1.0553e+00)	Acc@1  43.00 ( 47.29)	Acc@2  70.00 ( 77.06)
Epoch: [0][250/975]	Time  0.877 ( 0.880)	Data  0.000 ( 0.000)	Loss 1.0565e+00 (1.0543e+00)	Acc@1  48.00 ( 47.41)	Acc@2  70.00 ( 77.13)
Epoch: [0][300/975]	Time  0.877 ( 0.880)	Data  0.000 ( 0.000)	Loss 1.0413e+00 (1.0551e+00)	Acc@1  50.00 ( 47.22)	Acc@2  70.00 ( 77.02)
Epoch: [0][350/975]	Time  0.884 ( 0.880)	Data  0.000 ( 